In [ ]:
import matplotlib  
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
import lightkurve as lk
import scipy
import numpy as np

## Download light curve

In [ ]:
## Just download a random light curve available on MAST to test things out
obj = "WASP-100"
search_result = lk.search_lightcurve(obj, radius=5) # search radius 5 arcseconds
lc = search_result[-1].download() # downloads the most recent one (index -1, last one in table)

lc = lc.remove_nans() # Remove NaN values - I think they broke the code somehow?
time = lc.time.value
flux = lc.flux.value
flux_error = lc.flux_err.value

## Plot light curve
matplotlib.rcParams.update({'font.size': 16}) #Adjust font
matplotlib.rcParams['axes.linewidth'] = 2.0
f_lc, ax_lc = plt.subplots(figsize=(14, 5)) # Figure size (width, height)

ax_lc.scatter(time,flux, c="k", marker=".")
ax_lc.set(xlabel = "Time - 2457000 [BTJD days]", ylabel = "Flux [electrons/s]")

# Depending on the object you use, you may want to add a y-axis limit to see the
# data better


## Apply median filter

In [ ]:
## APPLY MEDIAN FILTER TO DATA
med = scipy.ndimage.median_filter(np.array(flux), size = 81) # (I don't know why I converted it to an array again here but I must've had some reason lol)

## Plot median filtered light curve with the data
matplotlib.rcParams.update({'font.size': 16}) # Set up plot
matplotlib.rcParams['axes.linewidth'] = 2.0
f_med, ax_med = plt.subplots(figsize=(14, 5))

x = np.linspace(min(time), max(time), len(med)) # x axis needs to have same number of points as y values or else it gets mad. Create a list of evenly
# spaced values from the min. time to max. time, number of values is the number of flux values

ax_med.scatter(time,flux,c="k",marker=".", label="Data")
ax_med.plot(x,med, label="Median Filter")

ax_med.set_ylim(np.median(flux)-0.01,np.median(flux)+0.005) # The transit in the data makes the median filter impossible to see lol, set axis limits 
# to a little bit outside the median of the data to zoom it in
ax_med.legend(fontsize=16,loc='lower right', framealpha=1); # Add legend

## Plot periodograms and find periods

In [ ]:
freq = np.linspace(1.0 / 40.0, 1.0 / 1.0, 5000)  # Frequency range
## Note: denominator of first number becomes maximum period it goes to

## PERIODOGRAM OF DATA
ls = LombScargle(time, flux) # x is the time from the LC without NaNs, y is the PDCSAP flux from lightkurve
power_ls = ls.power(freq, method="fast", normalization="psd")
power_ls /= len(time)

period_ls = 1.0 / freq[np.argmax(power_ls)] # Find period of data (corresponds to most significant peak)


## PERIODOGRAM OF MEDIAN FILTERED LIGHT CURVE
model = LombScargle(time, med) # med is flux after applying median filter
power_med = model.power(freq, method="fast", normalization="psd")
power_med /= len(time)

period_med = 1.0 / freq[np.argmax(power_med)] # Find period of median filtered data


# FIND DIFFERENCE BETWEEN THE TWO PERIODS
power_diff = power_ls - power_med
period_diff = 1.0 / freq[np.argmax(power_diff)]


# Print periods
print("\033[1m", "Period (Data): ", "\033[0m", str(period_ls), " days")
print("\033[1m", "Period (Median Filter): ", "\033[0m", str(period_med), " days")
print("\033[1m", "Period (Difference): ", "\033[0m", str(period_diff), " days")



## Plot data
matplotlib.rcParams.update({'font.size': 16}) ## Set up plot
matplotlib.rcParams['axes.linewidth'] = 2.0
f_per, ax_per = plt.subplots(figsize=(14, 5))

ax_per.plot(1.0 / freq, power_ls, c="k", label = "Data")
ax_per.plot(1.0 / freq, power_med, c="#00008b", label = "Median Filter", alpha=0.6)
ax_per.plot(1.0 / freq, power_diff, c="#8B0000", label = "Difference")

plt.axvline(period_ls, color="k", alpha=0.5) # Add vertical lines at significant periods
# You could do one at 2x the period too if you wanted?

ax_per.set_xlabel("Period [d]") # Add axis labels
ax_per.set_ylabel("LS Periodogram")
ax_per.set_xlim(0.0, 40.0) # Set x-axis limits from 0 to 40 days
ax_per.legend(fontsize=16,loc='upper right', framealpha=1); # Add legend
#plt.savefig("Peridogram.png", bbox_inches="tight")

## It'd probably be helpful to make a subplot of this figure showing just the difference, I was just lazy lol